# ИНСТРУМЕНТЫ СЕРИАЛИЗАЦИИ: PICKLE

Посмотрим, с помощью каких средств происходит сериализация объектов в Python.

В стандартную библиотеку Python входит модуль `pickle`, который служит для сериализации почти всех объектов произвольного типа.

Мы помним, что объекты находятся в оперативной памяти и направляются в байтовые потоки ввода-вывода. В байтовые потоки может быть направлен любой файлоподобный объект.

В ходе десериализации исходный объект воссоздаётся в оперативной памяти с теми же самыми значениями, но с новой идентичностью — новым адресом в памяти.

Для иллюстрации работы модуля `pickle` последовательно пройдёмся по всем шагам и разберёмся с тем, как сериализовать уже обученные алгоритмы машинного обучения.

## ШАГ №1

Обучим модель линейной регрессии на встроенном датасете о диабете — `Diabetes dataset`.

В данном датасете представлены десять исходных признаков: возраст, пол, индекс массы тела, среднее артериальное давление и шесть измерений сыворотки крови были получены для каждого из 442 пациентов с сахарным диабетом. Интерес представляет количественный показатель прогресса заболевания, замеренный через год после исходного измерения. Тип задачи — регрессия.

**Примечание.** Для простоты и наглядности мы опустим процесс предобработки данных, разведывательный анализ, разделение выборки на обучающую и тестовую, валидацию модели и подбор гиперпараметров, так как для наших целей это неважно. Мы уверены, что вы уже способны произвести эти этапы самостоятельно.

В качестве модели, прогнозирующей целевую переменную, возьмём простейшую линейную регрессию, и обучим её на исходных данных:

In [1]:
from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Инициализируем модель линейной регрессии
regressor = LinearRegression()
# Обучаем модель
regressor.fit(X,y)

## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

В результате выполнения кода получился объект класса `LinearRegression`, на который ссылается переменная `regressor`. При этом атрибуты объекта (веса модели линейной регрессии) были сформированы во время обучения. То есть объект `regressor` теперь является обученной моделью.

## ШАГ №2

Далее, когда мы получили обученную модель, нам необходимо сериализовать её, превратив объект Python в поток байтов. Для этого импортируем модуль pickle и воспользуемся функцией `dumps()`, в которую нужно передать объект Python.

In [2]:
import pickle

# Производим сериализацию обученной модели
model = pickle.dumps(regressor)

print(type(model))
print(type(regressor))
## bytes
## sklearn.linear_model._base.LinearRegression

<class 'bytes'>
<class 'sklearn.linear_model._base.LinearRegression'>


Как видим, мы создали объект model типа bytes.

## ШАГ №3

Давайте попробуем восстановить (десериализовать) объект Python. Для этого в модуле pickle есть функция `loads()`, в которую нужно передать сериализованный объект (поток байтов).

In [3]:
# Производим десериализацию
regressor_from_bytes = pickle.loads(model)
regressor_from_bytes
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

В результате десериализации мы смогли восстановить исходный объект (модель).

## ШАГ №4

Сохраним сериализованный объект прямо в файл. Для этого в pickle есть функция `dump()` (без s на конце). В неё необходимо передать имя файла или ссылку на открытый файл. Файл назовём `myfile`, его расширение — .pkl (формат данных pickle):

In [4]:
# Производим сериализацию и записываем результат в файл формата pkl
with open('myfile.pkl', 'wb') as output:
    pickle.dump(regressor, output)

Теперь у нас есть бинарный файл с готовой моделью, и мы можем передать его, например, ML-инженерам, которые будут заниматься деплоем модели на сервер.

## ШАГ №5

Посмотрим на код, который восстанавливает (десериализует) обученную модель из файла `myfile.pkl`. Для этого в pickle есть функция `load()` (без s на конце). В неё необходимо передать имя файла или ссылку на открытый файл.

In [5]:
# Производим десериализацию и извлекаем модель из файла формата pkl
with open('myfile.pkl', 'rb') as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

## ШАГ №6

Убедимся, что методы и результаты предсказаний обученной модели и модели, загруженной из файла, совпадают:

In [6]:
# Проверяем, что все элементы массивов предсказаний совпадают между собой
all(regressor.predict(X) == regressor_from_bytes.predict(X))
## True
all(regressor.predict(X) == regressor_from_file.predict(X))
## True

True

Как видите, исходная и восстановленная из байтов и файла модели дают одинаковые предсказания. Это значит, что теперь мы можем импортировать наши обученные модели в любое Python-приложение и пользоваться ими, минуя этап обучения и все этапы, предшествующие ему.

# ОГРАНИЧЕНИЯ

Как мы упоминали, у `pickle` есть ограничения. Например, мы не можем сериализовать лямбда-функции. Давайте посмотрим, что нам вернёт следующий код:

In [7]:
# my_lambda = lambda x: x*2
# with open('my_lambda.pkl', 'wb') as output:
#     pickle.dump(my_lambda, output)
 
##"PicklingError: Can't pickle <function <lambda>"

**Совет.** В таких случаях лучше пользоваться пакетом `dill`.

# СОХРАНЕНИЕ ПАЙПЛАЙНА

Ранее мы посмотрели простейший пример сериализации готовой модели.

У вас мог возникнуть вопрос: что делать, если перед подачей данных в модель их необходимо предобработать, например произвести стандартизацию, исключить неинформативные признаки? Неужели придётся прописывать все эти шаги в коде инференса модели? А что если вопросами инференса занимаются совершенно другие специалисты, которые вообще ничего не знают о машинном обучении и не умеют производить предобработку данных?

Конечно, мы должны передать результаты в таком виде, чтобы ими можно было воспользоваться без лишних манипуляций.

Мы уже упоминали, что `pickle` работает с любыми объектами Python. Поэтому для сохранения может быть доступна не просто обученная модель, но и целый пайплайн, включающий предобработку данных.

Например, мы хотим сериализовать пайплайн, который включает в себя min-max-нормализацию и отбор пяти наиболее важных факторов на основе корреляции Пирсона. Полученные в результате данные отправляются на вход модели линейной регрессии.

In [8]:
import pickle

from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)

# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x000002288DDA8940>)),
                ('Linear', LinearRegression())])

Пайплайн обучен. Давайте сохраним его в файл с помощью pickle:

In [9]:
# Сериализуем pipeline и записываем результат в файл
with open('my_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

Если сериализация завершилась успешно, то при инференсе модели мы сможем восстановить её из файла:

In [10]:
# Десериализуем pipeline из файла
with open('my_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

In [11]:
# Сравниваем предсказания исходного и восстановленного пайплайнов
print(all(pipe.predict(X) == loaded_pipe.predict(X)))

## True

True


**Примечание.** Если мы хотим сохранять сериализованные пайплайны в виде потока байтов, нужно использовать функции `dumps()` и `loads()`, а не `dump()` и `load()`.

Однако в процессе предобработки могут возникнуть шаги, которые нельзя реализовать стандартными методами sklearn. Например, для решения многих задач в нашем курсе мы часто использовали `feature engineering`, чтобы повысить качество работы моделей. Как встроить этот шаг в исходный пайплайн?

Для этого в sklearn можно организовать так называемые кастомные трансформеры. Такой трансформер должен наследоваться от двух классов: `TransformerMixin` и `BaseEstimator`.

Посмотрим на шаблон кастомного трансформера:
```

from sklearn.base import TransformerMixin, BaseEstimator
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''
 
    def __init__(self):
        '''
        Здесь прописывается инициализация параметров, не зависящих от данных.
        '''
        pass
 
    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''

        return self
 
    def transform(self, X):
        '''
        Здесь прописываются действия с данными.
        '''
        return X

```

У трансформера должно быть три обязательных метода:

`__init__()` — метод, который вызывается при создании объекта данного класса. Он предназначен для инициализации исходных параметров.
Например, у трансформера для создания полиномиальных признаков PolynomialFeatures из sklearn в методе __init__() параметр degree задаёт степень полинома.

`fit()` — метод, который вызывается для «обучения» трансформера. Он должен возвращать ссылку на сам объект (self).
Например, в трансформере StandardScaler в методе fit() прописано вычисление среднего значения и стандартного отклонения в каждом столбце таблицы, переданной в качестве параметра метода fit().

`transform()` — метод, который трансформирует приходящие на вход данные. Он должен возвращать преобразованный массив данных.
Например, при вызове метода transform() у StandardScaler из sklearn внутри происходит преобразование — вычитание из каждого столбца среднего и деление результата на стандартное отклонение. Причём среднее и стандартное отклонение вычисляются заранее в методе fit().

**Примечание.** Как мы знаем, у некоторых трансформеров из sklearn, например у того же `MinMaxScaler`, есть ещё и метод `fit_transform()`, который является комбинацией методов `fit()` и `transform()`.

Наш трансформер пока что ничего не делает. Предположим, мы хотим генерировать в данных новый признак, который является простым произведением первых трёх столбцов таблицы. Давайте пропишем в методе `transform()` эти действия.

Для работы такого трансформера нужны только исходные данные без дополнительных параметров, поэтому методы `__init__()` и `fit()` остаются без изменений.

In [12]:
from sklearn.base import TransformerMixin, BaseEstimator
import numpy as np


class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

Посмотрим, как работает наш кастомный трансформер. Создадим объект трансформера, вызовем метод `transform` и посмотрим на результирующий размер таблицы.

In [13]:
# Инициализируем объект класса MyTransformer (вызывается метод __init__)
custom_transformer = MyTransformer()
# Чисто формально вызываем метод fit, но у нас он ничего не делает
custom_transformer.fit(X)
# Трансформируем исходные данные (вызывается метод transform)
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

## Shape before transform: (442, 10)
## Shape after transform: (442, 11)

Shape before transform: (442, 10)
Shape after transform: (442, 11)


Видно, что в результате трансформации в исходную матрицу наблюдений добавился новый столбец.

Теперь давайте встроим этот трансформер в сам пайплайн — для этого достаточно добавить новый шаг в пайплайн.

In [14]:
# Создаём пайплайн, который включает Feature Engineering, нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('FeatureEngineering', MyTransformer()),
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x000002288DDA8940>)),
                ('Linear', LinearRegression())])

Наконец можно сериализовать полученный pipeline:

In [15]:
# Сериализуем pipeline и записываем результат в файл
with open('my_new_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

Теперь мы можем передать пайплайн и воспользоваться им для инференса, предварительно произведя десериализацию.

### Самостоятельная работа

Десериализуйте полученный pipeline с добавленным в него кастомной трансформации из файла. Затем предскажите значение целевой переменной для наблюдения, которое описывается следующим вектором:

`features = np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])`

В поле для ответа введите предсказанное значение целевой переменной, округлённое до целого числа.

In [16]:
features = np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])

In [17]:
# Десериализуем pipeline из файла
with open('my_new_pipeline.pkl', 'rb') as pkl_file:
    loaded_new_pipe = pickle.load(pkl_file)

In [18]:
print(loaded_new_pipe.predict(features).round())

[173.]


## БИБЛИОТЕКА JOBLIB

Как мы видим, pickle прекрасно справляется со своей задачей: мы можем сериализовать и восстанавливать любые Python-объекты, включая модели и даже пайплайны. Однако иногда массивы данных, на которых обучаются модели, бывают настолько большими, что после загрузки из pickle невозможно восстановить объект полностью.

В таких случаях вместо pickle лучше использовать библиотеку `joblib`. Этот модуль более эффективен и надёжен для работы с объектами, которые содержат большие массивы данных. Пожалуй, единственный минус этого модуля в том, что он может «консервировать» только в файл, поэтому вы не сможете получить объект в виде бинарной строки и работать с ним. В модуле попросту отсутствуют методы для работы с бинарной строкой. Формат файлов для сохранения — ``.joblib`.

В остальном работа с joblib полностью идентична работе с pickle: после обучения модели производим сериализацию с помощью функции `dump()`, а в коде самого приложения, где нужно использовать модель, выполняем десериализацию с помощью функции `load()`. В каждую из этих функций необходимо передать путь до файла для записи и чтения соответственно.

Для иллюстрации работы сохраним полученную линейную регрессию:

In [19]:
import joblib

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Обучаем модель линейной регрессии
regressor = LinearRegression()
regressor.fit(X, y)
# Производим сериализацию и сохраняем результат в файл формата .joblib
joblib.dump(regressor, 'regr.joblib')

## ['regr.joblib']

['regr.joblib']

In [20]:
#Загрузим файл заново (загрузка может быть произведена в другом файле с кодом):

# Десериализуем модель из файла
clf_from_jobliv = joblib.load('regr.joblib') 
# Сравниваем предсказания
all(regressor.predict(X) == clf_from_jobliv.predict(X))

## True

True

### Практика: pickle

В первом практическом юните этого модуля мы будем учиться работать с модулем pickle.

Ваш коллега Василий обучил модель и теперь просит вас проверить её на ваших данных. Он присылает вам pickle-файл (`model.pkl`). Загрузите модель, используя модуль `pickle`.

In [21]:
import pickle

with open('model.pkl', 'rb') as pkl_file:
    model = pickle.load(pkl_file)
    
  
#   secret word: skillfactory
# how is this possible? answer is here: https://youtu.be/xm-A-h9QkXg
# c:\Users\пользователь\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
# https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
#   warnings.warn(

secret word: skillfactory
how is this possible? answer is here: https://youtu.be/xm-A-h9QkXg


c:\Users\пользователь\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1. При загрузке вывелся секретный код. Введите его в поле ниже.
   
*Ответ:* secret word: skillfactory   

2. Проверьте, объект какого типа получился. Какую модель вам прислал коллега?
   
*Ответ:* LinearRegression  

In [22]:
model

LinearRegression(positive=True)

3. Теперь необходимо применить модель. Сделайте предсказание для следующего набора фичей: `[1, 1, 1, 0.661212487096872]`. Введите результат, предварительно округлив его до трёх знаков после точки-разделителя.

In [23]:
features = np.array([[1, 1, 1, 0.661212487096872]])
model.predict(features).round(3)[0]
#0.666

0.666

4. У присланной вам модели есть два поля (атрибута) с именами a и b. Создайте из них словарь с такими же именами ключей и значениями, а затем сохраните его в файл с помощью модуля pickle.

Чтобы вы могли проверить правильность решения задания, мы создали специальный проверочный скрипт. Скачайте его здесь.

Сохраните его рядом с вашим pickle-файлом (в той же папке) и запустите, передав первым аргументом имя файла. Если вы всё сделали правильно, на экран выведется ответ для следующего задания.

### КАК ЗАПУСТИТЬ СКРИПТ?

В ячейке Jupyter Notebook:
```
!python hw1_check_ol.py имя_pickle_файла.pkl
```

В терминале:
```
cd имя_папки_со_скриптом

python hw1_check_ol.py имя_pickle_файла.pkl
```


Если вы всё сделали правильно, в результате выполнения проверочного скрипта будет выведен секретный код.

Впишите полученный код в окно ниже:

После решения задания проверочный скрипт можно удалить — больше он вам не понадобится.

In [24]:
dict_model = {'a':model.a, 'b':model.b}

with open('model_practice.pkl', 'wb') as output:
    pickle.dump(dict_model, output)

In [25]:
!python hw1_check_ol.py model_practice.pkl
#('secret code 2:', '3c508')

('secret code 2:', '3c508')


Среда или требования к инференсу модели для вашего проекта могут быть устроены так, что потребуют реализации на языке программирования, отличном от Python. Например, если компания разрабатывает десктопное приложение, то для внедрения модели её потребуется «перевести» на Java или C++. Как это сделать?

# PREDICTIVE MODEL MARKUP LANGUAGE

В таких случаях используется генерация файла формата `PMML (Predictive Model Markup Language)`.

`PMML` — это XML-диалект, который применяется для описания статистических и DS-моделей. PMML-совместимые приложения позволяют легко обмениваться моделями данных между собой. Разработка и внедрение PMML осуществляется IT-консорциумом Data Mining Group.

Подробнее с PMML можно ознакомиться на официальном сайте.

К сожалению, далеко не все библиотеки для машинного обучения (в том числе sklearn) поддерживают возможность сохранения обученной модели в указанном формате. Однако для этого можно использовать сторонние библиотеки, и одной из самых популярных является Nyoka.

Давайте сохраним модель из предыдущего блока в формат PMML.

Для установки можно использовать систему управления пакетами pip:

`!pip3 install nyoka`

In [26]:
# Рассмотрим пример работы с библиотекой:

from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()['feature_names']

scaler = MinMaxScaler()
pipe = Pipeline([  
            ('Scaling', MinMaxScaler()),
            ('Linear', LinearRegression())
        ])
# Обучение пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipeline.pmml
skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="pipeline.pmml")

Итак, мы построили пайплайн обработки данных и обучили модель линейной регрессии. После этого мы с помощью функции `skl_to_pmml` сохранили модель в файл `pipe.pmml`.

Откройте файл `pipe.pmml` с помощью любого текстового редактора.

Давайте рассмотрим этот файл подробнее:

* Секция `<DataDictionary>` содержит информацию о признаках, включая наименование и тип данных, используемых для построения модели.
* Секция `<TransformationDictionary>` содержит информацию о необходимых преобразованиях для каждого признака. Обратите внимание, что в этом блоке также содержится информация для трансформации. Так как мы использовали minMaxScaler(), то в файле записаны минимальное и максимальное значения.

**Таким образом, в файле содержится вся информация для того, чтобы пайплайн можно было использовать на любом языке программирования.**

# OPEN NEURAL NETWORK EXCHANGE

В разработке моделей на основе нейронных сетей сегодня наиболее распространён формат `ONNX (Open Neural Network Exchange)`.

`ONNX (Open Neural Network Exchange)` — это открытый стандарт для обеспечения совместимости моделей машинного обучения. Он позволяет разработчикам искусственного интеллекта использовать модели с различными инфраструктурами, инструментами, средами исполнения и компиляторами.

Стандарт совместно поддерживается компаниями Microsoft, Amazon, Facebook и другими партнёрами как проект с открытым исходным кодом.

Часто стандарт ONNX и его библиотеки используют для конвертации из одного фреймворка в другой (например, из PyTorch в TensorFlow для использования в продакшене). Для конвертации различных фреймворков (не только DL) в формат ONNX и обратно существует ряд библиотек:

* ONNX-Tensorflow;
* Tensorflow-ONNX;
* Keras-ONNX;
* Sklearn-ONNX.
* и другие.
* 
Также в рамках стандарта ONNX есть инструмент `ONNX-runtime`. Он служит для ускорения инференса Python-моделей, а также инференса на других языках, например Java, C++.

In [28]:
# import onnxruntime as rt 
# from sklearn.datasets import load_boston
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from skl2onnx import convert_sklearn
# from skl2onnx.common.data_types import FloatTensorType


# # загружаем данные
# X, y = load_boston(return_X_y=True)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)
# print(X_train.shape, X_test.shape)

# # обучаем модель
# model = LinearRegression()
# model.fit(X_train, y_train)

# # делаем инференс моделью на тесте
# test_pred = model.predict(X_test)
# print('sklearn model predict:\n', test_pred)

# # конвертируем модель в ONNX-формат
# initial_type = [('float_input', FloatTensorType([None, 13]))]
# model_onnx = convert_sklearn(model, initial_types=initial_type)

# # сохраняем модель в файл
# with open("model.onnx", "wb") as f:
# 	f.write(model_onnx.SerializeToString())
 	 
# # Делаем инференс на тесте через ONNX-runtime
# sess = rt.InferenceSession("model.onnx")
# input_name = sess.get_inputs()[0].name
# label_name = sess.get_outputs()[0].name
# test_pred_onnx = sess.run([label_name],
#                 	{input_name:  X_test.astype(np.float32)})[0].reshape(-1)
# print('onnx model predict:\n',test_pred_onnx)